<a href="https://colab.research.google.com/github/TejoramV/CS6910_Assignment2/blob/main/Assignment2_Part-B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv1D, MaxPool1D, BatchNormalization, Dense, Flatten     
from tensorflow.keras.applications import InceptionV3, InceptionResNetV2, ResNet50, Xception
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy

!pip install wandb
import wandb
from wandb.keras import WandbCallback
from PIL import Image

     |████████████████████████████████| 1.8 MB 5.8 MB/s 
     |████████████████████████████████| 181 kB 17.9 MB/s 
     |████████████████████████████████| 144 kB 37.9 MB/s 
     |████████████████████████████████| 63 kB 663 kB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=f03076fbc959db48959738d48c4c0389bf5879a183ee2038fd5f157df048b8ac
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
def load_dataset(input_shape=(256,256)):

    train_dir = "/content/drive/MyDrive/inaturalist_12K/train"
    val_dir = "/content/drive/MyDrive/inaturalist_12K/val"
    train = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, shear_range=0.2, horizontal_flip=True)
    val = ImageDataGenerator(rescale=1./255)

    train_data = train.flow_from_directory(train_dir, target_size=input_shape, batch_size=256, shuffle=True)
    val_data = val.flow_from_directory(val_dir, target_size=input_shape, batch_size=256)    
    
    return train_data, val_data;

In [15]:
class buildModel(Model):    
    
    def __init__(self, base_model, image_size=(256, 256)):

        super(buildModel, self).__init__()

        self.base_model = self.pre_model(base_model, image_size)
        self.base_model.trainable=False

        self.conv1 = Conv1D(3, 12, 6, activation="relu")
        self.pool1 = MaxPool1D(3,3)
        self.b1 = BatchNormalization()
        self.conv2 = Conv1D(3, 6, 3, activation="relu")
        self.pool2 = MaxPool1D(3,3)
        self.b2 = BatchNormalization()
        self.output_layer = Dense(10, activation=None)
        
    @staticmethod
    def pre_model(name, image_size):
        
        image_size = list(image_size)
        image_size.append(3)
        
        input_shape = tuple(image_size) 

        if name=="InceptionV3":
            return InceptionV3(include_top=False, input_shape=input_shape, weights='imagenet')
        elif name=="InceptionResNetV2":
            return InceptionResNetV2(include_top=False, input_shape=input_shape, weights='imagenet')
        elif name=="ResNet50":
            return ResNet50(include_top=False, input_shape=input_shape, weights='imagenet')
        elif name=="Xception":
            return Xception(include_top=False, input_shape=input_shape, weights='imagenet')

    def call(self, x):

        x = Flatten()(self.base_model(x))
        
        x = tf.expand_dims(x, -1)
        x = self.b1(self.pool1(self.conv1(x)))
        x = self.b2(self.pool2(self.conv2(x)))
        x = Flatten()(x)

        return self.output_layer(x)

In [16]:
def train(image_size=(256, 256), epoch1=30, epoch2=10):
    lr = 0.0001
    config_defaults = {"base_model": "ResNet50"}

    wandb.init(config=config_defaults, project="CS6910_Assignment2-Part-B", magic=True)
    train_data, val_data = load_dataset(image_size)
    model = buildModel(wandb.config.base_model, image_size=image_size)
    model.compile(optimizer=RMSprop(learning_rate=lr), loss=CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    model.fit(train_data, validation_data=val_data, epochs=epoch1, callbacks=[WandbCallback()])
    
    pretrained_params = {"InceptionV3": 55, "InceptionResNetV2": 55, "ResNet50": 50, "Xception": 50}
    model.base_model.trainable = True
    freezing_point = len(model.base_model.layers) - pretrained_params[wandb.config.base_model]

    for layer in model.base_model.layers[:freezing_point]:
        layer.trainable =  False
    model.compile(optimizer=RMSprop(learning_rate=lr/10), loss=CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    model.fit(train_data, validation_data=val_data, epochs=epoch2, callbacks=[WandbCallback()])
    _, Val_accuracy = model.evaluate(val_data)
    wandb.log({"Val_Accuracy":Val_accuracy})

In [6]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
sweep_config = {
  'metric': {'name': 'Val_Accuracy','goal':'maximize'},
  "method": "grid",
  "parameters": {
        "base_model": {"values": ["InceptionV3", "InceptionResNetV2", "ResNet50", "Xception"]}
    }
}

sweep_id = wandb.sweep(sweep_config)

In [ ]:
wandb.agent(sweep_id, function=train)